In [54]:
import pandas as pd
import numpy as np

import sys
sys.path.insert(0, "../scripts")
import helpers as h
import window_detect2 as w
import scores as s
import ml_window as m
from icecream import ic

import json

import pickle

import plotly.graph_objects as go
from pprint import pprint

import plotly.graph_objects as go
# import kaleido
from plotly.subplots import make_subplots

In [2]:
a00, a01 = h.import_desired_data("A", "15T")
window_sched = np.array(a01["Window Open"])

In [3]:
with open('../constants/data_combos_230638.pkl', 'rb') as handle:
    test_dict = pickle.load(handle)

In [4]:
for k, v in test_dict.items():
    # print(k)
    obj = m.ML_Window_Detect(v["data"], window_sched)
    obj.run_all()
    test_dict[k]["accuracy"] = obj.accuracy
    test_dict[k]["choices"] = obj.choices

In [5]:
for k in test_dict.keys():
    choices = pd.Series(test_dict[k]["choices"])
    s1 = s.Scores(exp=a01, choices=choices, guess_times_ix=s.identify_changed_ix(choices))
    test_dict[k]["metrics"] = {}
    test_dict[k]["metrics"]["standard"] = s1.calc_standard_metrics()
    test_dict[k]["metrics"]["drdr"] = s1.calc_drdr_metrics()
    test_dict[k]["metrics"]["custom"] = s1.calc_hit_scores()

In [52]:
only_metrics = {k: v["metrics"] for k, v in test_dict.items()}
# only_metrics

In [47]:
short_metrics = {}
for k, v in test_dict.items():
    short_metrics[k] = {}

    short_metrics[k]["standard"] = {}

    short_metrics[k]["standard"]["macro avg f1-score"] = np.round(v["metrics"]["standard"]["macro avg"]["f1-score"],3)



    short_metrics[k]["custom"] = {}

    short_metrics[k]["custom"]["(hits + near hits)/guesses"] = v["metrics"]["custom"]["(hits + near hits)/guesses"]

    short_metrics[k]["custom"]["guesses/actions"] = v["metrics"]["custom"]["guesses/actions"]



    short_metrics[k]["drdr"] = {}

    short_metrics[k]["drdr"]["opening acc"] = v["metrics"]["drdr"]["bounded acc"]

    short_metrics[k]["drdr"]["unbounded opening acc"] = v["metrics"]["drdr"]["unbounded acc"]

    



In [40]:
def win_subplot(fig, key, col):
    fig.add_trace(go.Scatter(
        y=window_sched, 
        mode='lines',
        name="Window",
        legendgroup="1",
        opacity=0.5,
        showlegend=True, 
        line=dict(color="#636EFA")
    ), row = 1, col = col+1)


    fig.add_trace(go.Scatter(
        y=test_dict[key]["choices"], 
        mode='lines',
        name="Prediction",
        legendgroup="2",
        showlegend=True,
        line=dict(color="#EF553B")
    ), row = 1, col = col+1)

    return fig

In [41]:
titles = ["dt_meas+meas", "dt_amb+meas", "dt_meas+dt_amb"]

fig = make_subplots(rows=1, cols=3, shared_xaxes=True, subplot_titles=titles)

for ix, t in enumerate(titles): 
    fig = win_subplot(fig, t, ix)

fig.show()




In [48]:
b = sorted(short_metrics.items(), key=lambda x:x[1]["standard"]["macro avg f1-score"], reverse=True)
pprint(b[0:5], sort_dicts=False, width=20)

[('dt_meas+meas',
  {'standard': {'macro avg f1-score': 0.673},
   'custom': {'(hits + near hits)/guesses': 0.677,
              'guesses/actions': 2.385},
   'drdr': {'opening acc': 0.0,
            'unbounded opening acc': -10.14}}),
 ('dt_amb+meas',
  {'standard': {'macro avg f1-score': 0.647},
   'custom': {'(hits + near hits)/guesses': 0.589,
              'guesses/actions': 7.308},
   'drdr': {'opening acc': 0.0,
            'unbounded opening acc': -15.9}}),
 ('dt_meas+dt_amb',
  {'standard': {'macro avg f1-score': 0.641},
   'custom': {'(hits + near hits)/guesses': 0.449,
              'guesses/actions': 6.846},
   'drdr': {'opening acc': 0.0136,
            'unbounded opening acc': -15.28}}),
 ('dt_meas',
  {'standard': {'macro avg f1-score': 0.634},
   'custom': {'(hits + near hits)/guesses': 0.493,
              'guesses/actions': 5.769},
   'drdr': {'opening acc': 0.0,
            'unbounded opening acc': -14.53}}),
 ('dt_meas+dt_sim+dt_amb',
  {'standard': {'macro avg f1-s

In [46]:
titles = ["dt_amb", "dt_sim", "dt_meas+meas"]

fig = make_subplots(rows=1, cols=3, shared_xaxes=True, subplot_titles=titles)

for ix, t in enumerate(titles): 
    fig = win_subplot(fig, t, ix)

fig.show()

In [49]:
a = sorted(short_metrics.items(), key=lambda x:x[1]["custom"]["(hits + near hits)/guesses"]/x[1]["custom"]["guesses/actions"], reverse=True)
pprint(a[0:5], sort_dicts=False, width=20)

[('dt_amb',
  {'standard': {'macro avg f1-score': 0.303},
   'custom': {'(hits + near hits)/guesses': 0.273,
              'guesses/actions': 0.846},
   'drdr': {'opening acc': 0.0,
            'unbounded opening acc': -21.77}}),
 ('dt_sim',
  {'standard': {'macro avg f1-score': 0.275},
   'custom': {'(hits + near hits)/guesses': 0.667,
              'guesses/actions': 2.077},
   'drdr': {'opening acc': 0.0,
            'unbounded opening acc': -8.79}}),
 ('dt_meas+meas',
  {'standard': {'macro avg f1-score': 0.673},
   'custom': {'(hits + near hits)/guesses': 0.677,
              'guesses/actions': 2.385},
   'drdr': {'opening acc': 0.0,
            'unbounded opening acc': -10.14}}),
 ('meas+sim',
  {'standard': {'macro avg f1-score': 0.534},
   'custom': {'(hits + near hits)/guesses': 0.714,
              'guesses/actions': 2.692},
   'drdr': {'opening acc': 0.0,
            'unbounded opening acc': -14.56}}),
 ('amb',
  {'standard': {'macro avg f1-score': 0.574},
   'custom': {'(hi

In [43]:
a = sorted(short_metrics.items(), key=lambda x:x[1]["drdr"]["unbounded acc"], reverse=True)
pprint(a[0:5], sort_dicts=False, width=20)

[('dt_sim',
  {'standard': {'macro avg f1-score': 0.275},
   'custom': {'(hits + near hits)/guesses': 0.667,
              'guesses/actions': 2.077},
   'drdr': {'unbounded acc': -8.79}}),
 ('dt_meas+amb',
  {'standard': {'macro avg f1-score': 0.608},
   'custom': {'(hits + near hits)/guesses': 0.642,
              'guesses/actions': 4.077},
   'drdr': {'unbounded acc': -10.13}}),
 ('dt_meas+meas',
  {'standard': {'macro avg f1-score': 0.673},
   'custom': {'(hits + near hits)/guesses': 0.677,
              'guesses/actions': 2.385},
   'drdr': {'unbounded acc': -10.14}}),
 ('meas+amb',
  {'standard': {'macro avg f1-score': 0.605},
   'custom': {'(hits + near hits)/guesses': 0.568,
              'guesses/actions': 2.846},
   'drdr': {'unbounded acc': -11.66}}),
 ('sim',
  {'standard': {'macro avg f1-score': 0.442},
   'custom': {'(hits + near hits)/guesses': 0.613,
              'guesses/actions': 2.385},
   'drdr': {'unbounded acc': -11.9}})]


In [55]:
with open("../constants/svm_metrics_230629.json", 'w') as convert_file:
     convert_file.write(json.dumps(only_metrics, default=str))